# Preambles, import

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from qutip import*
from scipy.special import eval_hermite as hpoly
import sys
sys.path.append('/Users/longnguyen/Documents/GitHub/Fluxonium_berkeley/')
from Fluxonium_hamiltonians import Single_small_junction as fluxonium
from Fluxonium_hamiltonians import qchard_coupobj as coupobj
from Fluxonium_hamiltonians import qchard_transmon as transmon
from Fluxonium_hamiltonians import qchard_resonator as resonator
import plotting_settings
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 400

# Parameters, sweep coupling

In [ ]:
omega1 = 5.23 #GHz
omega2 = 5.29
omega3 = 5.46
omega4 = 5.63
alpha1 = -0.273
alpha2 = -0.273
alpha3 = -0.270
alpha4 = -0.267
nlev = 5
qubit1 = transmon.TransmonSimple(omega_q = omega1, alpha = alpha1, nlev = nlev)
qubit2 = transmon.TransmonSimple(omega_q = omega2, alpha = alpha2, nlev = nlev)
qubit3 = transmon.TransmonSimple(omega_q = omega3, alpha = alpha3, nlev = nlev)
qubit4 = transmon.TransmonSimple(omega_q = omega4, alpha = alpha4, nlev = nlev)

In [ ]:
print (omega1 + omega2)
print (omega1*2 + alpha1)
print (omega2*2 + alpha2)

## Direct coupling

In [ ]:
#Sweep coupling
J_C_array = np.linspace(0,0.005,101) #GHz
m_element = np.zeros_like(J_C_array)
energies_coupled_swE = np.zeros((qubit1.nlev * qubit2.nlev, len(J_C_array)))
for idx, J_C in enumerate(J_C_array):
    system = coupobj.CoupledObjects(qubit1, qubit2,
                                    [qubit1, qubit2, J_C, 'JC-rwa'])
    energies_coupled_swE[:,idx] = system.levels()
    m_element[idx] = abs(system.a(0).matrix_element(system.eigvec('00'), system.eigvec('01')))

In [ ]:
#Calculate ZZ rate:
plt.figure(figsize = [5,4])
zeta_ZZ = energies_coupled_swE[system.level_label('11'),:] - energies_coupled_swE[system.level_label('01'),:] \
- (energies_coupled_swE[system.level_label('10'), :] - energies_coupled_swE[system.level_label('00'), :])
freq_shift = energies_coupled_swE[system.level_label('01'), :] - energies_coupled_swE[system.level_label('10'), :]
freq_anchor = freq_shift[0]
freq_shift = freq_shift - freq_anchor
plt.plot(J_C_array*1e3, (freq_shift)*1e3, label = r'$\delta (f _{|01\rangle} - f_{|10\rangle})$')
plt.plot(J_C_array*1e3, (zeta_ZZ)*1e3, label = r'$\zeta_{ZZ}$', linewidth = 3.0, linestyle = '--')

plt.legend()
plt.tick_params(left=True, right=True,labelleft=True, labelright=True)
plt.ylabel('MHz')
plt.xlabel('$J$ (MHz)')
plt.axvline(x=2.5, linewidth = 1.0, linestyle = '--')
plt.axhline(y=0.13, linewidth = 1.0, linestyle = '--')
plt.xlim([J_C_array[0]*1e3, J_C_array[-1]*1e3])

In [ ]:
plt.plot(J_C_array*1e3, m_element)
plt.ylabel(r'$|\langle 00 |a \otimes I|01 \rangle | $')
plt.xlabel(r'$J$ (MHz)')

## Virtual coupling

In [ ]:
g_array = np.linspace(0,0.1,51)
m_element = np.zeros_like(g_array )
zz_array = np.zeros_like(g_array )
freq_shift = np.zeros_like(g_array)
omega_c = 6 # Resonator freq
coupl_resonator= resonator.Cavity(omega=omega_c, nlev=5)
for idx, g in enumerate(g_array):
    system = coupobj.CoupledObjects(qubit1, qubit2, coupl_resonator,
                                   [qubit1, coupl_resonator, g, 'JC-rwa'],
                                   [qubit2, coupl_resonator, g, 'JC-rwa'],
                                   [qubit1, qubit2,0, 'JC-rwa'])
    zz_array[idx] = system.level('110') + system.level('000') - system.level('100') - system.level('010')
    freq_shift[idx] = system.level('010') - system.level('100')
    m_element[idx] = abs(system.a(0).matrix_element(system.eigvec('000'), system.eigvec('010')))

In [ ]:
plt.plot(g_array*1e3, (freq_shift-freq_anchor)*1e3, label = r'$\delta (f _{|01\rangle} - f_{|10\rangle})$')
plt.plot(g_array*1e3, (zz_array)*1e3, label = r'$\zeta_{ZZ}$', linewidth = 3.0, linestyle = '--')
plt.legend()
plt.ylabel('MHz')
plt.xlabel('$g$ (MHz)')
# plt.axvline(x=2.5, linewidth = 1.0, linestyle = '--')
# plt.axhline(y=0.13, linewidth = 1.0, linestyle = '--')
plt.xlim([g_array[0]*1e3, g_array[-1]*1e3])

In [ ]:
plt.plot(g_array*1e3, m_element)
plt.ylabel(r'$|\langle 00 |a \otimes I|01 \rangle | $')
plt.xlabel(r'$g$ (MHz)')

## Multi-path

In [ ]:
J_C_array = np.linspace(0,0.01,51)
g = 0.08
m_element = np.zeros_like(g_array )
zz_array = np.zeros_like(g_array )
freq_shift = np.zeros_like(g_array)
omega_c = 7.2 # Resonator freq
coupl_resonator= resonator.Cavity(omega=omega_c, nlev=5)
for idx, J_C in enumerate(J_C_array):
    system = coupobj.CoupledObjects(qubit1, qubit2, coupl_resonator,
                                   [qubit1, coupl_resonator, g, 'JC-rwa'],
                                   [qubit2, coupl_resonator, g, 'JC-rwa'],
                                   [qubit1, qubit2, J_C, 'JC-rwa'])
    zz_array[idx] = system.level('110') + system.level('000') - system.level('100') - system.level('010')
    freq_shift[idx] = system.level('010') - system.level('100')
    m_element[idx] = abs(system.a(0).matrix_element(system.eigvec('000'), system.eigvec('010')))

In [ ]:
plt.plot(J_C_array*1e3, (freq_shift-freq_anchor)*1e3, label = r'$\delta (f _{|01\rangle} - f_{|10\rangle})$')
plt.plot(J_C_array*1e3, (zz_array)*1e3, label = r'$\zeta_{ZZ}$', linewidth = 3.0, linestyle = '--')
plt.legend()
plt.ylabel('MHz')
plt.xlabel('$J_C$ (MHz)')

In [ ]:
plt.plot(J_C_array*1e3, m_element)
plt.ylabel(r'$|\langle 00 |a \otimes I|01 \rangle | $')
plt.xlabel(r'$J_C$ (MHz)')

In [ ]:
w_c_array = np.linspace(4,12,41)
g = 0.06
J_C = 0.006
m_element = np.zeros_like(w_c_array)
zz_array = np.zeros_like(w_c_array)
freq_shift = np.zeros_like(w_c_array)

for idx, w_c in enumerate(w_c_array):
    coupl_resonator= resonator.Cavity(omega=w_c, nlev=5)
    system = coupobj.CoupledObjects(qubit1, qubit2, coupl_resonator,
                                   [qubit1, coupl_resonator, g, 'JC-rwa'],
                                   [qubit2, coupl_resonator, g, 'JC-rwa'],
                                   [qubit1, qubit2, J_C, 'JC-rwa'])
    zz_array[idx] = system.level('110') + system.level('000') - system.level('100') - system.level('010')
    freq_shift[idx] = system.level('010') - system.level('100')
    m_element[idx] = abs(system.a(0).matrix_element(system.eigvec('000'), system.eigvec('010')))

In [ ]:
plt.plot(w_c_array, (freq_shift-freq_anchor)*1e3, label = r'$\delta (f _{|01\rangle} - f_{|10\rangle})$')
plt.plot(w_c_array, (zz_array)*1e3, label = r'$\zeta_{ZZ}$', linewidth = 3.0, linestyle = '--')
plt.legend()
plt.ylim([-10,10])
plt.ylabel('MHz')
plt.xlabel('r$\omega_r$ (GHz)')

In [ ]:
plt.plot(w_c_array, m_element)
plt.ylabel(r'$|\langle 00 |a \otimes I|01 \rangle | $')
plt.xlabel(r'$\omega_r$ (GHz)')

# SQUID
## Parameters

In [ ]:
import sys
sys.path.append('/Users/longnguyen/Documents/GitHub/Fluxonium_berkeley/')
from Fluxonium_hamiltonians import qchard_squid as squid
import numpy as np
E_C = 0.2
E_Jsum = 22
d = 0.05
phi_ext = 0.0
nlev = 7
nlev_charge = 40
coupler = squid.Squid(E_C = E_C, E_Jsum = E_Jsum, d = d, phi_ext = phi_ext, nlev = nlev, nlev_charge = nlev_charge)

In [ ]:
print(coupler.levels())

In [ ]:
print(coupler.n_ij(0,1))

In [ ]:
phi_ext_array = np.linspace(0,1,101)*np.pi*2
energies = np.zeros((len(phi_ext_array), nlev))
for idx, phi_ext in enumerate (phi_ext_array):
    coupler = squid.Squid(E_C = E_C, E_Jsum = E_Jsum, d = d, phi_ext = phi_ext, nlev = nlev, nlev_charge = nlev_charge)
    energies[idx, :] = coupler.levels()

In [ ]:
for idx in range (1, nlev):
    plt.plot(phi_ext_array/np.pi/2, energies[:,idx] - energies[:,0])
plt.ylim([0,10])